In [1]:
!pip install -q streamlit openai-whisper pydub nltk keybert sentence-transformers
!apt-get install -y ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [2]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
%%writefile app.py
import streamlit as st
import whisper
import tempfile
import os
import numpy as np
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

st.set_page_config(layout="wide")

st.title("🎙️ AI Podcast Transcription Studio")

# ---------------- LOAD MODELS ----------------
@st.cache_resource
def load_models():
    model = whisper.load_model("tiny")  # fast + stable
    kw = KeyBERT('all-MiniLM-L6-v2')
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    return model, kw, embedder

whisper_model, kw_model, embedder = load_models()

# ---------------- SIDEBAR ----------------
st.sidebar.title("🎧 Podcast Control")

option = st.sidebar.radio(
    "Choose Audio Source:",
    ["Use Default Podcast", "Upload Podcast"]
)

audio_path = None


# ✅ DEFAULT PODCAST
if option == "Use Default Podcast":

    if os.path.exists("podcast1.mp3"):
        audio_path = "podcast1.mp3"
        st.sidebar.success("Podcast ready ✅")
    else:
        st.sidebar.warning("Upload podcast1.mp3 into Colab files.")


# ✅ BULLETPROOF UPLOADER
else:

    uploaded = st.sidebar.file_uploader(
        "Upload audio",
        type=["mp3","wav","m4a"]
    )

    if uploaded is not None:

        suffix = uploaded.name.split(".")[-1]

        temp_file = tempfile.NamedTemporaryFile(
            delete=False,
            suffix=f".{suffix}"
        )

        temp_file.write(uploaded.read())
        audio_path = temp_file.name

        st.sidebar.success("Upload successful ✅")


# ---------------- TRANSCRIPTION ----------------
def transcribe_with_timestamps(file):

    result = whisper_model.transcribe(
        file,
        fp16=False   # prevents tensor crash
    )

    return result


# ---------------- SEGMENT GROUPING ----------------
def build_segments(result):

    segments = result["segments"]

    grouped = []
    current_text = ""
    start_time = segments[0]["start"]

    for seg in segments:

        current_text += seg["text"]

        # group every ~60 seconds
        if seg["end"] - start_time > 60:
            grouped.append({
                "start": start_time,
                "text": current_text
            })

            start_time = seg["end"]
            current_text = ""

    grouped.append({
        "start": start_time,
        "text": current_text
    })

    return grouped


# ---------------- KEYWORDS ----------------
def extract_keywords(text):
    kws = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1,2),
        stop_words='english',
        top_n=5
    )
    return [k[0] for k in kws]


# ---------------- SUMMARY ----------------
def summarize(text):
    return " ".join(text.split(".")[:2])


# ---------------- FORMAT TIME ----------------
def format_time(seconds):
    mins = int(seconds // 60)
    secs = int(seconds % 60)
    return f"{mins:02}:{secs:02}"


# ---------------- MAIN UI ----------------
if audio_path:

    st.audio(audio_path)

    if st.button("🚀 Generate AI Insights"):

        with st.spinner("AI is analyzing your podcast..."):

            result = transcribe_with_timestamps(audio_path)

        st.success("Transcription Complete!")

        segments = build_segments(result)

        st.header("📊 Smart Podcast Chapters")

        for i, seg in enumerate(segments):

            time_label = format_time(seg["start"])
            keywords = extract_keywords(seg["text"])
            summary = summarize(seg["text"])

            with st.expander(f"🧠 Chapter {i+1} — {time_label}"):

                st.markdown(f"### ⏱ Start Time: **{time_label}**")

                # ⭐ Jump Trick
                st.audio(audio_path, start_time=int(seg["start"]))

                st.markdown("### 📌 Summary")
                st.write(summary)

                st.markdown("### 🔑 Keywords")
                st.write(", ".join(keywords))

                st.markdown("### 📝 Text")
                st.write(seg["text"])

        st.header("📄 Full Transcript")
        st.write(result["text"])

else:
    st.info("👈 Select or upload a podcast from the sidebar.")


Writing app.py


In [4]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


--2026-02-17 04:23:06--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb [following]
--2026-02-17 04:23:07--  https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/4fddf4d7-e02d-44dc-9e5a-ef9e28afdd54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-17T05%3A12%3A22Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&

In [ ]:
!streamlit run app.py &>/content/logs.txt &
!cloudflared tunnel --url http://localhost:8501


2026-02-17T04:23:09Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-02-17T04:23:09Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-02-17T04:23:14Z INF +--------------------------------------------------------------------------------------------+
2026-02-17T04:23:14Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-02-17T04:23:14Z INF |  https://meets-education-rrp-alcohol.trycloudflare.com